# Integration Pipeline - Soft Skills Analysis

This notebook combines all analyzers into a unified pipeline for real-time soft skills evaluation.

## Goals
1. Combine fluency, gaze, gesture, and posture analyzers
2. Create unified scoring system
3. Benchmark real-time performance
4. Export complete pipeline for backend integration

In [ ]:
import os
import sys
from pathlib import Path

# Add project root to path
PROJECT_ROOT = Path(os.getcwd()).parent.parent.parent
sys.path.insert(0, str(PROJECT_ROOT))

import cv2
import numpy as np
import time
import mediapipe as mp
from collections import deque
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Tuple

print(f"Project root: {PROJECT_ROOT}")

## 1. Import Utility Functions

In [ ]:
# Import from training utils (generated by previous notebooks)
from ml.softskills.training import fluency_utils
from ml.softskills.training import gaze_utils
from ml.softskills.training import gesture_utils
from ml.softskills.training import posture_utils

print("Imported utility modules from training/")

## 2. Unified Data Classes

In [ ]:
@dataclass
class VisualMetrics:
    """Visual analysis results from a single frame."""
    # Eye contact
    eye_contact_score: float = 0.0
    gaze_direction: str = 'unknown'
    is_looking_at_camera: bool = False
    
    # Hand gestures
    gesture_score: float = 0.0
    hands_visible: bool = False
    num_hands: int = 0
    
    # Posture
    posture_score: float = 0.0
    is_upright: bool = True
    shoulders_level: bool = True
    
    # Face detection
    face_detected: bool = False
    body_detected: bool = False


@dataclass
class AudioMetrics:
    """Audio/speech analysis results."""
    # Fluency
    fluency_score: float = 0.0
    wpm: float = 0.0
    filler_count: int = 0
    fillers_detected: List[str] = field(default_factory=list)
    
    # Grammar (future)
    grammar_score: float = 0.0


@dataclass
class SoftSkillsScore:
    """Final combined soft skills score."""
    overall_score: float = 0.0
    
    # Individual scores
    fluency_score: float = 0.0
    grammar_score: float = 0.0
    eye_contact_score: float = 0.0
    gesture_score: float = 0.0
    posture_score: float = 0.0
    expression_score: float = 0.0
    
    # Weights used
    weights: Dict[str, float] = field(default_factory=lambda: {
        'fluency': 0.30,
        'grammar': 0.20,
        'eye_contact': 0.15,
        'gesture': 0.10,
        'posture': 0.10,
        'expression': 0.10
    })
    
    def calculate_overall(self):
        """Calculate weighted overall score."""
        self.overall_score = (
            self.fluency_score * self.weights['fluency'] +
            self.grammar_score * self.weights['grammar'] +
            self.eye_contact_score * self.weights['eye_contact'] +
            self.gesture_score * self.weights['gesture'] +
            self.posture_score * self.weights['posture'] +
            self.expression_score * self.weights['expression']
        )
        return self.overall_score

print("Data classes defined")

## 3. Unified Pipeline Class

In [ ]:
class SoftSkillsPipeline:
    """
    Unified pipeline for real-time soft skills analysis.
    
    Combines:
    - Eye contact detection (MediaPipe Face Mesh)
    - Hand gesture analysis (MediaPipe Hands)
    - Posture estimation (MediaPipe Pose)
    - Speech fluency analysis (text-based)
    """
    
    def __init__(self, fps: int = 30, enable_tracking: bool = True):
        self.fps = fps
        self.enable_tracking = enable_tracking
        
        # Initialize MediaPipe components
        self.mp_face_mesh = mp.solutions.face_mesh
        self.mp_hands = mp.solutions.hands
        self.mp_pose = mp.solutions.pose
        
        # Create detectors
        self.face_mesh = self.mp_face_mesh.FaceMesh(
            max_num_faces=1,
            refine_landmarks=True,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )
        
        self.hands = self.mp_hands.Hands(
            static_image_mode=False,
            max_num_hands=2,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )
        
        self.pose = self.mp_pose.Pose(
            static_image_mode=False,
            model_complexity=1,
            smooth_landmarks=True,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )
        
        # Trackers for temporal analysis
        if enable_tracking:
            self.hand_tracker = gesture_utils.HandMovementTracker(fps=fps)
            self.posture_tracker = posture_utils.PostureTracker()
        else:
            self.hand_tracker = None
            self.posture_tracker = None
        
        # Aggregated scores
        self.score_history = deque(maxlen=fps * 60)  # 1 minute of scores
        
        print("SoftSkillsPipeline initialized")
    
    def process_frame(self, frame: np.ndarray) -> VisualMetrics:
        """
        Process a single video frame for visual metrics.
        
        Args:
            frame: BGR image (numpy array)
        
        Returns:
            VisualMetrics with all visual analysis results
        """
        # Convert to RGB once
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        h, w = frame.shape[:2]
        
        metrics = VisualMetrics()
        
        # 1. Gaze/Eye Contact Analysis
        face_results = self.face_mesh.process(rgb_frame)
        if face_results.multi_face_landmarks:
            metrics.face_detected = True
            landmarks = face_results.multi_face_landmarks[0].landmark
            
            # Calculate gaze
            _, _, avg_ratio = gaze_utils.calculate_gaze_ratio(landmarks, w, h)
            metrics.gaze_direction = gaze_utils.get_gaze_direction(avg_ratio)
            
            # Simple head pose estimation (using nose and eye positions)
            # For full implementation, use the estimate_head_pose from notebook
            head_yaw = 0  # Simplified
            head_pitch = 0
            
            score_result = gaze_utils.calculate_eye_contact_score(avg_ratio, head_yaw, head_pitch)
            metrics.eye_contact_score = score_result['overall_score']
            metrics.is_looking_at_camera = score_result['is_looking_at_camera']
        
        # 2. Hand Gesture Analysis
        hand_results = self.hands.process(rgb_frame)
        left_center = None
        right_center = None
        
        if hand_results.multi_hand_landmarks:
            for hand_landmarks, handedness in zip(
                hand_results.multi_hand_landmarks,
                hand_results.multi_handedness
            ):
                label = handedness.classification[0].label
                center = gesture_utils.get_hand_center(hand_landmarks, w, h)
                
                if label == 'Left':
                    left_center = center
                else:
                    right_center = center
            
            metrics.hands_visible = True
            metrics.num_hands = len(hand_results.multi_hand_landmarks)
        
        # Update hand tracker
        if self.hand_tracker:
            self.hand_tracker.update(left_center, right_center)
            tracker_metrics = self.hand_tracker.get_metrics()
            velocity = max(
                tracker_metrics['left_hand'].get('velocity', 0),
                tracker_metrics['right_hand'].get('velocity', 0)
            )
            stability = tracker_metrics['avg_stability']
        else:
            velocity = 50  # Default
            stability = 80
        
        gesture_result = gesture_utils.calculate_gesture_score(
            metrics.hands_visible,
            metrics.num_hands,
            stability,
            velocity
        )
        metrics.gesture_score = gesture_result['overall_score']
        
        # 3. Posture Analysis
        pose_results = self.pose.process(rgb_frame)
        if pose_results.pose_landmarks:
            metrics.body_detected = True
            
            shoulder_metrics = posture_utils.calculate_shoulder_metrics(
                pose_results.pose_landmarks, w, h
            )
            lean_metrics = posture_utils.calculate_body_lean(
                pose_results.pose_landmarks, w, h
            )
            
            # Update posture tracker
            if self.posture_tracker:
                self.posture_tracker.update(
                    shoulder_metrics['center'],
                    shoulder_metrics['tilt_angle'],
                    lean_metrics['lean_angle']
                )
                posture_stability = self.posture_tracker.calculate_stability()['overall_stability']
            else:
                posture_stability = 80
            
            posture_result = posture_utils.calculate_posture_score(
                shoulder_metrics['tilt_angle'],
                lean_metrics['lean_angle'],
                posture_stability
            )
            metrics.posture_score = posture_result['overall_score']
            metrics.is_upright = lean_metrics['is_upright']
            metrics.shoulders_level = shoulder_metrics['is_level']
        
        return metrics
    
    def process_audio(self, transcript: str, duration_seconds: float) -> AudioMetrics:
        """
        Process speech transcript for audio metrics.
        
        Args:
            transcript: Speech transcript text
            duration_seconds: Audio duration
        
        Returns:
            AudioMetrics with fluency analysis
        """
        metrics = AudioMetrics()
        
        if not transcript or duration_seconds <= 0:
            return metrics
        
        fluency_result = fluency_utils.calculate_fluency_score(
            transcript, duration_seconds
        )
        
        metrics.fluency_score = fluency_result['overall_score']
        metrics.wpm = fluency_result['wpm']
        metrics.filler_count = fluency_result['filler_count']
        metrics.fillers_detected = fluency_result['fillers_detected']
        
        # Grammar score (placeholder - requires NLP integration)
        metrics.grammar_score = 75.0  # Default
        
        return metrics
    
    def calculate_final_score(
        self,
        visual_metrics: VisualMetrics,
        audio_metrics: AudioMetrics
    ) -> SoftSkillsScore:
        """
        Calculate final weighted soft skills score.
        """
        score = SoftSkillsScore(
            fluency_score=audio_metrics.fluency_score,
            grammar_score=audio_metrics.grammar_score,
            eye_contact_score=visual_metrics.eye_contact_score,
            gesture_score=visual_metrics.gesture_score,
            posture_score=visual_metrics.posture_score,
            expression_score=75.0  # Placeholder
        )
        score.calculate_overall()
        return score
    
    def close(self):
        """Release resources."""
        self.face_mesh.close()
        self.hands.close()
        self.pose.close()

print("SoftSkillsPipeline class defined")

## 4. Performance Benchmark

In [ ]:
def benchmark_pipeline(num_frames: int = 100, frame_size: Tuple[int, int] = (640, 480)):
    """
    Benchmark pipeline performance.
    
    Target: 5 FPS (200ms per frame) for real-time analysis.
    """
    pipeline = SoftSkillsPipeline(enable_tracking=True)
    
    # Create synthetic test frames
    test_frame = np.random.randint(0, 255, (frame_size[1], frame_size[0], 3), dtype=np.uint8)
    
    times = []
    
    print(f"Benchmarking with {num_frames} frames at {frame_size}...")
    
    for i in range(num_frames):
        start = time.time()
        metrics = pipeline.process_frame(test_frame)
        elapsed = time.time() - start
        times.append(elapsed)
    
    pipeline.close()
    
    avg_time = np.mean(times) * 1000  # ms
    max_time = np.max(times) * 1000
    min_time = np.min(times) * 1000
    fps = 1000 / avg_time
    
    print(f"\n{'='*50}")
    print(f"Performance Results:")
    print(f"{'='*50}")
    print(f"Average time per frame: {avg_time:.1f} ms")
    print(f"Min: {min_time:.1f} ms, Max: {max_time:.1f} ms")
    print(f"Estimated FPS: {fps:.1f}")
    print(f"")
    print(f"Target: 5 FPS (200ms/frame)")
    if avg_time <= 200:
        print(f"✅ PASS - Real-time capable")
    else:
        print(f"⚠️ NEEDS OPTIMIZATION - Consider reducing model complexity")
    
    return {
        'avg_ms': avg_time,
        'fps': fps,
        'pass': avg_time <= 200
    }

# Run benchmark (note: will be slow without GPU on synthetic data)
# benchmark_pipeline(50)

## 5. Complete Usage Example

In [ ]:
def demo_soft_skills_analysis():
    """
    Demo the complete soft skills analysis pipeline.
    """
    print("Soft Skills Analysis Demo")
    print("=" * 50)
    
    # Initialize pipeline
    pipeline = SoftSkillsPipeline(fps=30, enable_tracking=True)
    
    # Simulate video frames (in real use, from webcam)
    # For demo, create a blank frame
    dummy_frame = np.zeros((480, 640, 3), dtype=np.uint8)
    
    # Process some frames
    print("\nProcessing 10 frames...")
    for i in range(10):
        visual_metrics = pipeline.process_frame(dummy_frame)
    
    # Simulate speech transcript
    transcript = """
    Well, um, I think I would be a good fit for this position because, 
    you know, I have experience in software development and, like, 
    I really enjoy solving complex problems.
    """
    duration = 15.0  # seconds
    
    # Process audio
    audio_metrics = pipeline.process_audio(transcript, duration)
    
    # Calculate final score
    final_score = pipeline.calculate_final_score(visual_metrics, audio_metrics)
    
    print("\n" + "=" * 50)
    print("FINAL SOFT SKILLS SCORE")
    print("=" * 50)
    print(f"\n🎯 Overall Score: {final_score.overall_score:.1f}/100")
    print("\nBreakdown:")
    print(f"  📣 Fluency:     {final_score.fluency_score:.1f} (weight: {final_score.weights['fluency']*100:.0f}%)")
    print(f"  📝 Grammar:     {final_score.grammar_score:.1f} (weight: {final_score.weights['grammar']*100:.0f}%)")
    print(f"  👁️  Eye Contact: {final_score.eye_contact_score:.1f} (weight: {final_score.weights['eye_contact']*100:.0f}%)")
    print(f"  🤚 Gestures:    {final_score.gesture_score:.1f} (weight: {final_score.weights['gesture']*100:.0f}%)")
    print(f"  🧍 Posture:     {final_score.posture_score:.1f} (weight: {final_score.weights['posture']*100:.0f}%)")
    print(f"  😊 Expression:  {final_score.expression_score:.1f} (weight: {final_score.weights['expression']*100:.0f}%)")
    
    print("\nFluency Details:")
    print(f"  WPM: {audio_metrics.wpm:.0f}")
    print(f"  Fillers detected: {audio_metrics.filler_count}")
    print(f"  Filler words: {audio_metrics.fillers_detected}")
    
    pipeline.close()
    
    return final_score

# Run demo
# demo_soft_skills_analysis()

## 6. Export Pipeline for Backend

In [ ]:
# The pipeline class will be exported to backend in Phase 4
# See: backend/ai-service/app/services/softskills_pipeline.py

print("""
Pipeline Export Notes:
======================

The SoftSkillsPipeline class should be adapted for the backend:

1. File: backend/ai-service/app/services/softskills_pipeline.py
2. Use FastAPI WebSocket for real-time frame streaming
3. Add async processing support
4. Integrate with PostgreSQL for storing results
5. Add Redis caching for session state

See implementation_plan.md for full details.
""")

## 7. Summary

### Pipeline Components

| Component | Input | Output | Weight |
|-----------|-------|--------|--------|
| Fluency Analyzer | Transcript + Duration | WPM, Fillers, Score | 30% |
| Grammar Analyzer | Transcript | Error count, Score | 20% |
| Eye Contact | Video Frame | Gaze direction, Score | 15% |
| Gesture Analyzer | Video Frame | Hand visibility, Score | 10% |
| Posture Analyzer | Video Frame | Lean, Stability, Score | 10% |
| Expression Analyzer | Video Frame | (Future) | 10% |

### Performance Targets
- **Frame processing**: < 200ms (5 FPS)
- **Latency**: < 500ms end-to-end
- **Memory**: < 500MB GPU memory

### Next Steps
1. Create database migration (Phase 3)
2. Implement backend services (Phase 4)
3. Integrate with frontend WebSocket (Phase 5)
4. Add grammar analysis using LanguageTool
5. Add expression/emotion detection